In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
img_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
txt_model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=1000)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
import numpy as np
import pandas as pd
device = torch.device('cuda')

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

In [ ]:
path = '/content/gdrive/MyDrive/da/open'

In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/da/open/train.csv')

In [ ]:
train_txt = train_data['overview']
train_cat3 = train_data['cat3']
train_imgpath = train_data['img_path']

In [ ]:
img = train_imgpath.apply(lambda x: path+x[1:])

In [ ]:
# train_images = []
# for i in tqdm(range(len(img))):
#     train_images.append(Image.open(img.iloc[i]))

 98%|█████████▊| 16626/16986 [2:58:40<03:52,  1.55it/s]


FileNotFoundError: ignored

In [ ]:
for i in tqdm(range(16626, len(img))):
    train_images.append(Image.open(img.iloc[i]))

100%|██████████| 360/360 [04:01<00:00,  1.49it/s]


In [ ]:
len(train_images)

16986

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_cat3)
train_target = label_encoder.transform(train_cat3)

In [ ]:
len(label_encoder.classes_)

In [ ]:
class SampleData(Dataset):
    def __init__(self, X, img, target, tokenizer, feature_extractor):
        super(SampleData, self).__init__()
        self.X = X.to_numpy()
        self.img = img
        
        self.target = target

    def __len__(self):
        return len(self.target)

    def __getitem__(self, item):
        encoding = tokenizer(self.X[item],
                                  return_token_type_ids = True,
                                  return_attention_mask = True,
                                  truncation = True,
                                  padding='max_length',
                                  max_length = 250,
                                  return_tensors = 'pt')
        img = feature_extractor(images=self.img[item], return_tensors="pt")
        return encoding['input_ids'][0], encoding['attention_mask'][0], img['pixel_values'][0], self.target[item]

In [ ]:
train_data = SampleData(train_txt, train_images, train_target, tokenizer, feature_extractor)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
class ClassifierModel(nn.Module):
    def __init__(self, txt_model, img_model):
        super(ClassifierModel, self).__init__()
        self.txt_model = txt_model
        self.img_model = img_model

        self.to_dense_img = nn.Linear(1000, 256)
        self.to_dense_txt = nn.Linear(1000, 256)

        self.classfier = nn.Linear(512, 128)
        

    def forward(self, x, mask, img):
        img = self.img_model(img)[0]
        # img = nn.Dropout(p=0.1)(img)
        img = F.relu(self.to_dense_img(img))

        txt = self.txt_model(x, mask)[0]
        # txt = nn.Dropout(p=0.1)(txt)
        txt = F.relu(self.to_dense_txt(txt))

        concat = torch.cat([img, txt], dim=1)
        x = self.classfier(concat)
        return x

In [ ]:
model = ClassifierModel(txt_model , img_model).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
epochs = 20

In [ ]:
from tqdm import tqdm

In [39]:
losses = []
accuracies = []

for i in range(epochs):
  print('\n'+'===='*5+str(i+1)+'epoch '+'===='*5+'\n')
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0


  for input_ids_batch, attention_masks_batch, y_img_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_masks_batch.to(device), y_img_batch.to(device))

        
    loss = nn.functional.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("\nBatch Loss:", total_loss, "Accuracy:", correct.float() / total, '\n')
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)


====================1epoch ====================



 19%|█▉        | 100/531 [01:57<07:14,  1.01s/it]


Batch Loss: 436.91025257110596 Accuracy: tensor(0.1850, device='cuda:0') 



 38%|███▊      | 200/531 [03:48<05:41,  1.03s/it]


Batch Loss: 774.5830852985382 Accuracy: tensor(0.2458, device='cuda:0') 



...


Batch Loss: 4.354492981103249 Accuracy: tensor(0.9987, device='cuda:0') 



 94%|█████████▍| 500/531 [06:25<00:23,  1.30it/s]


Batch Loss: 5.926561609492637 Accuracy: tensor(0.9986, device='cuda:0') 



100%|██████████| 531/531 [06:49<00:00,  1.30it/s]

Train Loss: 6.3669087359448895 Accuracy: tensor(0.9985, device='cuda:0')


In [40]:
torch.save(model.state_dict, '/content/gdrive/MyDrive/da/open/concat_electra_vit.pt')

In [41]:
test_data = pd.read_csv('/content/gdrive/MyDrive/da/open/test.csv')

In [42]:
test_txt = test_data['overview']
test_imgpath = test_data['img_path']

In [43]:
!unzip -qq /content/gdrive/MyDrive/da/open/image/test.zip

In [44]:
path = '/content'

In [45]:
test_img = test_imgpath.apply(lambda x: path+x[7:])
test_images = []
for i in tqdm(range(len(test_img))):
    test_images.append(Image.open(img.iloc[i]))

100%|██████████| 7280/7280 [00:06<00:00, 1135.88it/s]


In [46]:
test_dt = SampleData(test_txt,test_images, np.zeros(len(test_txt), dtype=np.int32), tokenizer, feature_extractor)

In [47]:
test_ld = DataLoader(test_dt, shuffle=False)

In [48]:
with torch.no_grad():
    result = []    
    
    for a,b,c,_ in tqdm(test_ld):
        model.eval()

        re = model(a.to(device), b.to(device), c.to(device))
        result.append(torch.max(re, axis=1)[1].to('cpu').item())

100%|██████████| 7280/7280 [05:12<00:00, 23.29it/s]


In [49]:
result_ = pd.Series(result).apply(lambda x: label_encoder.classes_[x]).values

In [ ]:
prev_state = pd.read_csv('/content/gdrive/MyDrive/da/open/submit_koelectra.csv')

In [54]:
submit_df = pd.read_csv('/content/gdrive/MyDrive/da/open/sample_submission.csv')
submit_df['cat3'] = result_
submit_df.to_csv('/content/gdrive/MyDrive/da/open/concat_electra_vit.csv', index=False)

In [ ]:
torch.save(model.state_dict, '/content/gdrive/MyDrive/da/open/concat_electra_vit.pt')